In [1]:
import os
import cv2
import random
# import shutil
import numpy as np
import PIL
from matplotlib import pyplot as plt
from copy import deepcopy
import serial
import time
import pandas as pd
import json

## Functions

In [ ]:
calib = json.load(open("calib.json"))
calib[0]

In [ ]:
configuration = ['*'] * 54

configuration[4] = 'w'
configuration[13] = 'r'
configuration[22] = 'b'
configuration[31] = 'o'
configuration[40] = 'g'
configuration[49] = 'y'


In [ ]:
cap_rects = [(285, 160, 305, 170), #corner
             (310, 185, 325, 205), #corner
             (275, 185, 290, 205), #corner
             (310, 230, 325, 250), #edge
             (275, 230, 290, 250)] #edge

In [ ]:
def draw_guide(frame):
    global cap_rects
    line_clr = (255,0,255) #pink
    rect_clr = (255,0,255) #pink
    rect_thick = 2
    
    cv2.line(frame,(300, 180),(300,315),line_clr,2)
    cv2.line(frame,(300, 180),(100,100),line_clr,2)
    cv2.line(frame,(300, 180),(500,85),line_clr,2)
    
    for r in cap_rects:
        cv2.rectangle(frame, r[:2], r[2:], line_clr,rect_thick)


In [ ]:
def cap_frame(show_guide=True):
    cap = cv2.VideoCapture(0)

    while(True):
        ret, frame = cap.read()
        if(ret):
            if show_guide:
                guide_frame = deepcopy(frame)
                draw_guide(guide_frame)
            cv2.imshow("output", guide_frame)
            if cv2.waitKey(1) & 0xFF == ord('c'):
                break
        else:
            break
        
    cv2.destroyAllWindows()
    cap.release()
    return frame

In [ ]:
def capture(cells):
    global cap_rects
    ret = []
    cap = cv2.VideoCapture(0)
    rd = False
    while(not rd):
        rd, img = cap.read()
    
    
    for i in cells:
        cell = img[cap_rects[i][1]:cap_rects[i][3], cap_rects[i][0]: cap_rects[i][2]]
        # cell = rgb2hsv(cell)
        mean = (np.mean(cell, axis=(0, 1)).round().astype(int)).tolist()
        ret.append(mean)
    return(ret)

In [ ]:
capture([0])

In [ ]:
cap_frame();

In [ ]:
caps = capture([0, 1, 2, 3, 4])
caps

In [ ]:
def solve(pos, colour):
    pos_req = calib[pos]
    closest_d = 1000000000
    closest_c = None
    # print(colour)
    # print(pos_req)
    # print(pos_req.items())
    
    for clr, val in pos_req.items():
        # print(colour[0])
        # print(val[0])
        dist = (colour[0]-val[0])*(colour[0]-val[0]) + (colour[1]-val[1])*(colour[1]-val[1]) + (colour[2]-val[2])*(colour[2]-val[2])
        # print(dist,clr)
        if dist < closest_d:
            # print("asgn", dist,clr)

            closest_d = dist
            closest_c = clr

    # print("st",closest_c)
    # print("dist",closest_d)
    print(pos, closest_c)
    return(closest_c)


In [ ]:
solve(0, caps[0])


In [ ]:
# configuration[33], configuration[0], configuration[38], configuration[3], configuration[41] = 
solve(0, caps[0]), solve(1, caps[1]), solve(2, caps[2]), solve(3, caps[3]), solve(4, caps[4]) 


In [ ]:
    configuration[33], configuration[0], configuration[38], configuration[3], configuration[41] = solve(0, capture([0])), solve(1, capture([1])), solve(2, capture([2])), solve(3, capture([3])), solve(4, capture([4])) 


In [ ]:
def saveconf(inds):
    caps = capture([0, 1, 2, 3, 4])
    for i,j in enumerate(inds):
        configuration[j]= caps[i]
        

In [ ]:
def cube_scan():
    _ = cap_frame()
    delay = 1

    calib = json.load(open("calib.json"))


    configuration = ['*'] * 54

    configuration[4] = 'w'
    configuration[13] = 'r'
    configuration[22] = 'b'
    configuration[31] = 'o'
    configuration[40] = 'g'
    configuration[49] = 'y'


    def solve(pos, colour):
        pos_req = calib[pos]
        closest_d = 1000000000
        closest_c = None

        for clr, val in pos_req.items():
            dist = (colour[0]-val[0])*(colour[0]-val[0]) + (colour[1]-val[1])*(colour[1]-val[1]) + (colour[2]-val[2])*(colour[2]-val[2])
            if dist < closest_d:
                closest_d = dist
                closest_c = clr

        # print(pos, closest_c)
        return(closest_c)

    
    def saveconf(inds, msg):
        caps = capture([0, 1, 2, 3, 4])
        for i, j in enumerate(inds):
            if j < 0:
                continue
            configuration[j]= caps[i]
        sendserial(msg, delay=delay)
        
    
    # FRONT LAYER:
    saveconf([33, 0, 38, 3, 41], "f")
    saveconf([44, 6, 9, 7, 10], "f")
    saveconf([11, 8, 24, 5, 21], "f")
    saveconf([18, 2, 35, 1, 34], "f")


    # BACK LAYER:
    sendserial("ddrrlluu", delay=delay)
    saveconf([15, 45, 42, 48, 39], "f")
    saveconf([36, 51, 27, 46, 16], "f")
    saveconf([29, 53, 20, 50, 23], "f")
    saveconf([26, 47, 17, 52, 28], "f")
    sendserial("uullrrddl", delay=delay)

    # EDGES:
    saveconf([-1, -1, -1, 30, 37], "ll")
    saveconf([-1, -1, -1, 12, 43], "luDf")
    saveconf([-1, -1, -1, 25, 14], "ff")
    saveconf([-1, -1, -1, 19, 32], "fUd")

    final = ""
    for i in configuration:
        final += i
    final
    return(final)





In [27]:
a = ['a','b']
b = ''

In [28]:
for i in range(len(a)):
    b+=a[len(a)-i-1]

In [15]:
b = b[:-1]

In [29]:
b

'ba'

In [ ]:
for i in [0,1,2,3]:
    print(i)
    if i == 2:
        continue
    print(i)

In [ ]:
 def saveconf(inds, msg):
            for i, j in enumerate(inds):
                if j < 0:
                    continue
                print (i, j)
            # print(msg, delay=delay)

In [ ]:
# FRONT LAYER:
saveconf([33, 0, 38, 3, 41], "f")
saveconf([44, 6, 9, 7, 10], "f")
saveconf([11, 8, 24, 5, 21], "f")
saveconf([18, 2, 35, 1, 34], "f")


# BACK LAYER:
self.sendserial("ddrrlluu", delay=delay)
saveconf([15, 45, 42, 48, 39], "f")
saveconf([36, 51, 27, 46, 16], "f")
saveconf([29, 53, 20, 50, 23], "f")
#self.sendserial("F", delay=delay)
# print(np.array([[self.capture_cells([0])]]))
# self.sendserial("f", delay=delay)
# plt.imshow(np.array([self.capture_cells([0])]))
# plt.show()
saveconf([26, 47, 17, 52, 28], "f")
self.sendserial("uullrrddl", delay=delay)

# EDGES:
saveconf([-1, -1, -1, 30, 37], "ll")
saveconf([-1, -1, -1, 12, 43], "luDf")
saveconf([-1, -1, -1, 25, 14], "ff")
saveconf([-1, -1, -1, 19, 32], "fUd")

In [ ]:
conf = ['w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'y', 'y', 'y', 'y', 'y', 'y', 'y', 'y', 'y']

def saveconf(inds, msg):
    caps = self.capture_cells([0, 1, 2, 3, 4])
    for i, j in enumerate(inds):
        if j < 0:
            continue
        configuration[j] = solve(i, caps[i])
    print(msg)


# FRONT LAYER:
saveconf([33, 0, 38, 3, 41], "f")
saveconf([44, 6, 9, 7, 10], "f")
saveconf([11, 8, 24, 5, 21], "f")
saveconf([18, 2, 35, 1, 34], "f")


# BACK LAYER:
print("ddrrlluu")
saveconf([15, 45, 42, 48, 39], "f")
saveconf([36, 51, 27, 46, 16], "f")
saveconf([29, 53, 20, 50, 23], "f")
#print("F", delay=delay)
# print(np.array([[self.capture_cells([0])]]))
# print("f", delay=delay)
# plt.imshow(np.array([self.capture_cells([0])]))
# plt.show()
saveconf([26, 47, 17, 52, 28], "f")
print("uullrrddl")

# EDGES:
saveconf([-1, -1, -1, 30, 37], "ll")
saveconf([-1, -1, -1, 12, 43], "luDf")
saveconf([-1, -1, -1, 25, 14], "ff")
saveconf([-1, -1, -1, 19, 32], "fUd")